# Generate Training Data

In order to train the discriminative models, we need training data. To this end, we create synthetic training data via a simple simulation of kicks on goal. The algorithm works as follows:

- Sample a position (i.e. $x \in [0, 1]$ and $y \in [0, 2]$)
- Sample from one of the possible ranges `{wide-left`, `block-left`, `block-center`, `block-right`, `wide-right}`
- Sample from one the possible powers `{weak`, `strong}`
- Kick!
- Record each sample

### 1D Deterministic Data Generator

In [4]:
import math
import random

import itertools
from collections import namedtuple

State = namedtuple('State', ['pos', 'vel', 'label'])
World = namedtuple('World', ['goal', 'end'])

def data_generator(world, num_starts=3, num_ends=5, time_steps=10, decay=.8):
    """Generate data from a fixed set of grid points
    
    Parameters
    ----------
    world : World
    The dimensions of the world
    
    num_starts : int
    The number of starting points you want
    
    num_ends : int
    The number of ending points you want
    
    time_steps : int
    The number of time steps for the ball to get to the robot. This number should be calculated based on
    the assumption that we're operating at 50 Hz. A reasonable number of time steps would thus be 100 (2 seconds)
    
    decay : float
    The constant decay rate of the velocity of the ball
    
    """
    goal, end = world
    
    # Starting and ending points
    starts, ends = np.linspace(0, goal/2, num_starts), np.linspace(goal/2, end, num_ends)
    
    for start, end in itertools.product(starts, ends):
        # Starting velocities to reach the goal
        c = (1-decay) / (1-decay**time_steps)
        dstart = c * (end-start)

        # Calculate how far you've traveled from the start at each time_step
        decays = [0] + [decay**pow for pow in range(time_steps)]

        # Velocity at each time step
        velocities = dstart*np.array(decays)

        # Distance traveled at each time step
        deltas = np.cumsum(velocities)

        # Compute the actual points given distances
        positions = start + deltas
        
        # Make sure the first state is always False
        yield State(positions[0], velocities[0], False)

        # Yield the points one by one
        for position, velocity in zip(positions[1:], velocities[1:]):
            yield State(position, velocity, end>=2)


data_points = data_generator(World(goal=2, end=3), num_starts=2, num_ends=3)

### Taking Invisible Points into Account

In [5]:
data_points = list(data_points)

pd.DataFrame(data_points, columns=State._fields).groupby('label').size()

label
False    26
True     40
dtype: int64

### Filter Away Invisible Points

In [6]:
data_points = [data_point for data_point in data_points if not data_point.pos > 2]

data_points_1d = pd.DataFrame(data_points, columns=State._fields)

data_points_1d.groupby('label').size()

label
False    26
True     26
dtype: int64

In [7]:
%store data_points_1d

Stored 'data_points_1d' (DataFrame)
